In [ ]:
Sicher, ich kann ein neues Jupyter Notebook mit allen Code-Snippets und Erläuterungen für Sie erstellen. Hier ist das neue Notebook `Schlafqualitaet_Projektbericht.ipynb`:

```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
```

# 1. Einleitung

In diesem Projekt analysieren wir einen umfassenden Datensatz zu Schlaf- und Gesundheitsmetriken, um Erkenntnisse über die Faktoren zu gewinnen, die die Schlafqualität beeinflussen. Wir werden die Daten bereinigen, transformieren und verschiedene Analysen wie Clustering, Regression und maschinelles Lernen durchführen, um Muster und Zusammenhänge zu identifizieren.

# 2. Beschreibung des "Comprehensive Sleep and Health Metrics" Datensatzes

## 2.1 Überblick über den Datensatz

```python
df = pd.read_csv('../data/final_dataset_with_normalized_and_standardized_features.csv')
print(f"Der Datensatz enthält {df.shape[0]} Zeilen und {df.shape[1]} Spalten.")
```

## 2.2 Variablenbeschreibung

Der Datensatz enthält folgende Variablen:

- `Heart_Rate_Variability`: Variabilität der Herzfrequenz
- `Movement_During_Sleep`: Bewegung während des Schlafes
- `Sleep_Duration_Hours`: Schlafdauer in Stunden
- `Sleep_Quality_Score`: Bewertung der Schlafqualität (Zielvariable)
- `Caffeine_Intake_mg`: Koffeinaufnahme in Milligramm
- ...

# 3. Datenbereinigung und -aufbereitung

## 3.1 Behandlung fehlender Werte

```python
# Fehlende Werte mit dem Median ersetzen
imputer = SimpleImputer(strategy='median')
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
```

## 3.2 Ausreißeranalyse

```python
# Ausreißer identifizieren und entfernen
z_scores = np.abs(stats.zscore(df_imputed))
df_clean = df_imputed[(z_scores < 3).all(axis=1)]
```

## 3.3 Feature Engineering

```python
# Neue Spalte "Light_Sleep_Ratio" erstellen
df_clean['Light_Sleep_Ratio'] = df_clean['Light_Exposure_hours'] / df_clean['Sleep_Duration_Hours']
```

## 3.4 Normalisierung und Standardisierung

```python
# Normalisieren und Standardisieren der Features
scaler_standard = StandardScaler()
scaler_minmax = MinMaxScaler()

normalized_features = scaler_minmax.fit_transform(df_clean.drop(columns=['Sleep_Quality_Score']))
standardized_features = scaler_standard.fit_transform(df_clean.drop(columns=['Sleep_Quality_Score']))

df_final_normalized = pd.DataFrame(normalized_features, columns=[col + '_normalized' for col in df_clean.columns if col != 'Sleep_Quality_Score'])
df_final_standardized = pd.DataFrame(standardized_features, columns=[col + '_standardized' for col in df_clean.columns if col != 'Sleep_Quality_Score'])

df_final_combined = pd.concat([df_clean, df_final_normalized, df_final_standardized], axis=1)
```

# 4. Explorative Datenanalyse (EDA)

## 4.1 Univariate Analyse

```python
# Histogramme für numerische Variablen
df_final_combined.hist(figsize=(12, 8))
plt.show()
```

## 4.2 Bivariate Analyse

```python
# Streudiagramm für zwei Variablen
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Caffeine_Intake_mg', y='Sleep_Quality_Score', data=df_final_combined)
plt.show()
```

## 4.3 Multivariate Analyse

```python
# Paarplot für mehrere Variablen
sns.pairplot(df_final_combined[['Sleep_Quality_Score', 'Caffeine_Intake_mg', 'Sleep_Duration_Hours', 'Movement_During_Sleep']])
plt.show()
```

## 4.4 Korrelationsanalyse

```python
# Korrelationsmatrix
corr_matrix = df_final_combined.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.show()
```

## 4.5 Visualisierungen

```python
# Beispielvisualisierung: Boxplot für Schlafqualität nach Koffeinaufnahme-Kategorien
df_final_combined['Caffeine_Category'] = pd.cut(df_final_combined['Caffeine_Intake_mg'], bins=[0, 100, 200, np.inf], labels=['Niedrig', 'Mittel', 'Hoch'])
plt.figure(figsize=(8, 6))
sns.boxplot(x='Caffeine_Category', y='Sleep_Quality_Score', data=df_final_combined)
plt.show()
```

# 5. Methodik

## 5.1 Clusteranalyse

```python
# K-Means Clustering
kmeans = KMeans(n_clusters=3, random_state=42)
cluster_labels = kmeans.fit_predict(df_final_combined.drop(columns=['Sleep_Quality_Score']))
df_final_combined['Cluster'] = cluster_labels

# Visualisierung der Cluster
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Caffeine_Intake_mg', y='Sleep_Quality_Score', hue='Cluster', data=df_final_combined)
plt.show()
```

## 5.2 Regressionsmodelle

```python
# Lineare Regression
from sklearn.linear_model import LinearRegression

X = df_final_combined.drop(columns=['Sleep_Quality_Score'])
y = df_final_combined['Sleep_Quality_Score']

lin_reg = LinearRegression()
lin_reg.fit(X, y)

print(f"Lineare Regression R^2: {lin_reg.score(X, y):.2f}")
```

## 5.3 Maschinelles Lernen

```python
# Random Forest Regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

rf_reg = RandomForestRegressor(random_state=42)
scores = cross_val_score(rf_reg, X, y, cv=5, scoring='r2')

print(f"Random Forest Regression durchschnittliches R^2: {scores.mean():.2f}")
```

## 5.4 Modellvalidierung und -evaluation

```python
# Modellvalidierung mit Kreuzvalidierung
from sklearn.model_selection import KFold, cross_val_score

kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Lineare Regression
lin_scores = cross_val_score(lin_reg, X, y, cv=kf, scoring='r2')
print(f"Lineare Regression durchschnittliches R^2 (Kreuzvalidierung): {lin_scores.mean():.2f}")

# Random Forest Regression
rf_scores = cross_val_score(rf_reg, X, y, cv=kf, scoring='r2')
print(f"Random Forest Regression durchschnittliches R^2 (Kreuzvalidierung): {rf_scores.mean():.2f}")
```